In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
item_df = pd.read_csv('item_dict.csv', delimiter = ',')
item_dict = dict(zip(item_df.en_label.values, [np.fromstring(i, dtype=int, sep=',') for i in item_df.item_id.values]))
del item_df
print(item_dict['tesla'])

[    9036   163343   478214  1050485  1428953  1548225  2406220  7705502
 16258100 27701406]
Wall time: 11.3 s


In [3]:
print(len(item_dict))

6230693


In [4]:
%%time
#unidirection will take ~2.5 min
#bidirectional will take ~5 min
graph_df = pd.read_csv('statements_uni.csv', delimiter = ',')
graph_dict = dict(zip(graph_df.Source.values, [np.fromstring(i, dtype=int, sep=',') for i in graph_df.Targets.values]))
del graph_df
print(graph_dict[9036])

[  367211       60  6581097 16086586 12750167       30    28513   131964
        5    31519   689775  1905051    43035   101333     9301   169470
   205375  1326886  1906857  2267418   632404  4920135   613201    93728
   111734   678263  1332315  1366018  1517792  3332215  5442484 17552952
 41799198   127885  1280677   131566  1132636  1132636    12152       60
       90     1085     1781     3617    13298    49258   367211 37251206
 15501913    93996  6374306  3739104   602358  2041543 12912667 17378135
 67311526    38104      150      188      397      652     1860     9056
     9067     9301    83364 16098713  5460604  1700481     1860]
Wall time: 1min 15s


In [5]:
def traverse(root, levels=1):
    level = [np.unique(graph_dict.get(root))]
    
    for i in range(1, levels):
        temp = np.unique(np.concatenate([graph_dict.get(j, np.array([], dtype=int)) for j in level[i-1]]))
        level.append(np.setdiff1d(temp, np.concatenate(level)))
    
    return level

#for i in range(1, 11): 
#    print(f'{i} levels')
#    %time test = traverse(9036, i)
#    print(len(test[-1]), 'nodes')

In [6]:
def Traverse(root, levels=1):
    
    if (type(graph_dict.get(root))==None):
        a =[]
        return a
    level = [np.unique(graph_dict.get(root))]
    #print (type(level))
    for i in range(1, levels):
        try:
            temp = np.unique(np.concatenate([graph_dict.get(j, np.array([], dtype=int)) for j in level[i-1]]))
        except:
            return []
        level.append(np.setdiff1d(temp, np.concatenate(level)))
    for i in range(len(level)):
        level[i]=set(level[i])
    return level



In [7]:
a=Traverse(9036,10)

In [8]:
len(a)

10

In [9]:
    
def find_shortest_path_length(src,dst,maxlev,Dict):
    if dst==-1 or src ==-1:
        return 0
    if (src,dst) in Dict: return Dict[(src,dst)]
    else:
    
        pathlength=0
        found=False
        reachend=False
        level_sets=Traverse(src,maxlev)
        if level_sets==[]:
            Dict[(src,dst)]=5
            return 5
        level=0
    
        while ((not found) and (not reachend)):
            if (level==maxlev):
                reachend=True
        
            elif dst in level_sets[level]:
                pathlength=level
                found =True
            else:
                level+=1
        if found == True:
            Dict[(src,dst)]=level+1
            return level+1
        else:
            Dict[(src,dst)]=5
            return 5
        

In [10]:
%%time
find_shortest_path_length(9036,5507494,10, {})

Wall time: 4.11 s


10

In [11]:
find_shortest_path_length(9036,-1,10,{})

0

In [12]:
sentences = pickle.load(open("sentences.p", "rb"))

In [13]:
sentences[0]

([[275102],
  [1675432, 56538769, 30272204],
  [18366588, 217, 209754],
  [],
  [],
  [7913, 2175775, 57477615],
  [3627558, 2497107],
  [13211, 26245],
  [4135847, 4135849, 4135859, 7128194, 16562681, 17536409, 31283669, 11692785],
  [809, 385059, 22061936, 36982750, 57477613],
  []],
 array(['voivode', 'iuga', 'moldavia', '1400)(known', 'iurg', 'romanian',
        'yury', 'ruthenian', 'jerzy', 'polish', 'ologul'], dtype=object),
 ('Voivode', 149044, 275102))

In [1]:
len(sentences)

NameError: name 'sentences' is not defined

In [16]:
sentences[4][0]

[[48797348, 65807361, 35622],
 [776675, 48798668, 73002, 1343033, 27939009, 27939181, 30293377]]

In [17]:
sentences[4][2]

('SmackDown', 55778545, 48797348)

In [14]:
#pairwise for all => takes extremely long
def disambiguate(candidate_lists):
    Dict={}
    for i in range(len(candidate_lists)):
        if len(candidate_lists[i])==0:
            candidate_lists[i].append(-1)
    #print (candidate_lists)
    combination_list = list(itertools.product(*candidate_lists))
    #print(len(combination_list))
    disambiguation = combination_list[0]
    least = math.inf
    for combinations in combination_list:
        #print("dict is currently:")
        #print(Dict)
        distance_sum = 0
        #generate a list of pairs of candidates to calculate pairwise distances
        #print("current combination:", combinations)
        pairs = list(itertools.combinations(combinations,2))
        #print(pairs)
        #print(len(pairs))
        for pair in pairs:
            #print("current pair:", pair)
            pair_dist = find_shortest_path_length(pair[0],pair[1],3,Dict)
            #print("pair dist", pair_dist)
            distance_sum += pair_dist
        if (distance_sum < least):
            least = distance_sum
            disambiguation = combinations
        #print(least)
        #print("end of current combination==========")
    return disambiguation

In [34]:
sentences[4][2]

('SmackDown', 55778545, 48797348)

In [15]:
%%time
import itertools
import math
disambiguate(sentences[4][0])

Wall time: 95.8 ms


(65807361, 73002)

In [19]:
def total_count(nested_list):
    count =0
    for lists in nested_list:
        count = count + len(lists)
    return count

def prune_sentences(sentences):
    for sentence in sentences[:]:
        #print(sentence[0])
        sentence_length = len(sentence[0])
        total_disamb_count = total_count(sentence[0])
        if (len(sentence[0][0])==1 or sentence_length > 10 or total_disamb_count > 50):
            sentences.remove(sentence)


In [20]:
copy = list(sentences)
prune_sentences(copy)

In [21]:
len(copy)

33388

## Disambiguation

In [23]:
%%time
import itertools
import math

def disamb_sentences(sentences):
    cor_count=0
    for sentence in sentences:
    
        if (disambiguate(sentence[0])[0]==sentence[2][2]):
            cor_count = cor_count+1
    return cor_count/len(sentences)

disamb_sentences(copy[0:10000])

Wall time: 1h 1min 26s


0.7452

https://arxiv.org/abs/1802.01074
http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.557.136
https://www.kaggle.com/gabrielaltay/kdwd-subclass-path-ner


In [29]:
%%time
import itertools
import math
from multiprocessing import Pool


def process_image(sentence):
    cor_count=0
    if (disambiguate(sentence[0])[0]==sentence[2][2]):
        cor_count = cor_count+1
    return cor_count

if __name__ == '__main__':
    pool = Pool(os.cpu_count()-1)
    pool.map(process_image, sentences)  
    
disambiguate(sentences[0][0])

[[275102], [1675432, 56538769, 30272204], [18366588, 217, 209754], [-1], [-1], [7913, 2175775, 57477615], [3627558, 2497107], [13211, 26245], [4135847, 4135849, 4135859, 7128194, 16562681, 17536409, 31283669, 11692785], [809, 385059, 22061936, 36982750, 57477613], [-1]]
4320
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<cla

(275102, 30272204, 217, -1, -1, 7913, 3627558, 13211, 17536409, 809, -1)